In [6]:
import requests
from tqdm import tqdm
import time
from bs4 import BeautifulSoup
import pandas as pd

def numStr(num):
    if num >= 10:
        return str(num)
    else:
        return '0' + str(num)

Base = "http://race.sp.netkeiba.com/?pid=race_result&race_id="
dst = ''
df_col = ['year', 'date', 'field', 'race', 'race_name'
          , 'course', 'head_count', 'rank', 'horse_name'
          , 'gender', 'age', 'trainerA', 'trainerB', 'weight', 'c_weight', 'jackie', 'j_weight'
          , 'odds','popu']

s = time.time()
key = 0

for year in range(2008, 2022):
    df = pd.DataFrame()
    dic = {}
    for i in range(1, 11):
        for j in range(1, 11):
            for k in range(1, 11):
                for l in range(1, 13):
                    # urlでぶっこ抜く
                    e = time.time()
                    print(f'{year}, {i}, {j}, {k}, {l}')
                    url = Base + str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l)
                    sleep = max(1 - e + s, 0)
                    time.sleep(sleep)
                    s = time.time()
                    while 1:
                        try:
                            html = requests.get(url)
                            break
                        except:
                            print('SLEEP!!!!')
                            time.sleep(5)
                            continue
                    html.encoding = 'EUC-JP'

                    # scraping
                    soup = BeautifulSoup(html.text, 'html.parser')
                    # ページがあるかの判定
                    if soup.find_all('div', attrs={'class', 'Result_Guide'})!=[]:
                        break
                    else:
                        #共通部分を抜き出す
                        CommonYear = year
                        CommonDate = soup.find_all('div', attrs={'class', 'Change_Btn'})[0].contents[1].contents[0]
                        CommonField= soup.find_all('div', attrs={'class', 'Change_Btn Course'})[0].string.strip()
                        CommonRace = soup.find_all('div', attrs={'Race_Num'})[0].span.string
                        CommonRname= soup.find_all('dt', attrs={'class', 'Race_Name'})[0].contents[0].strip()
                        CommonCourse= soup.find_all('dd', attrs={'Race_Data'})[0].span.string
                        CommonCourseDistance = soup.find_all('dd', attrs={'Race_Data'})[0].contents[2].contents[0]
                        CommonTime = soup.find_all('dd', attrs={'Race_Data'})[0].contents[0]
                        CommonWeather = soup.find_all('dd', attrs={'Race_Data'})[0].contents[4].contents[0]
                        CommonHcount= soup.find_all('dd', attrs={'class', 'Race_Data'})[0].contents[3].split()[-1]
                        CommonCond = soup.find_all('dd', attrs={'Race_Data'})[0].contents[-1].contents[0]
                        CommonTurn = soup.find_all('dd', attrs={'Race_Data'})[0].contents[3][2]
                        try:
                            money_list = soup.find_all('div', attrs={'class', 'RaceList_Item01'})[1].contents[1].contents[2][4:-2].split('、')
                        except:
                            money_list = soup.find_all('div', attrs={'class', 'RaceList_Item01'})[1].contents[1].contents[1][4:-2].split('、')
                        goal_time = soup.find_all('td', attrs=['class', 'Time'])
                        rank = soup.find_all('div', attrs='Rank')
                        horse_name = soup.find_all('dt', attrs=['class', 'Horse_Name'])
                        detail_left = soup.find_all('span', attrs=['class', 'Detail_Left'])
                        detail_right = soup.find_all('span', attrs=['class', 'Detail_Right'])
                        odds = soup.find_all('td', attrs=['class', 'Odds'])
                        waku = soup.find_all('td', attrs=['class', 'Num'])
                        for m in range(len(soup.find_all('div', attrs='Rank'))):
                            if isinstance(rank[m].contents[0], str):
                                continue
                            dst = pd.Series(index=df_col)
                            try:
                                dst['race_num'] = str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l)
                                dst['year'] = CommonYear
                                dst['date'] = CommonDate
                                dst['field']= CommonField #開催場所
                                dst['race'] = CommonRace
                                dst['race_name'] = CommonRname
                                dst['course'] = CommonCourse
                                dst['distance'] = CommonCourseDistance
                                dst['time'] = CommonTime
                                dst['cond'] = CommonCond
                                dst['turn'] = CommonTurn
                                if m == 0:
                                    dst['goal_time'] = goal_time[0].contents[0]
                                else:
                                    dst['goal_time'] = goal_time[m].contents[1].contents[1].contents[0]
                                dst['weather'] = CommonWeather
                                dst['head_count'] = CommonHcount #頭数
                                dst['rank'] = rank[m].contents[0]
                                dst['horse_name'] = horse_name[m].a.string
                                detailL = detail_left[m]
                                dst['gender'] = list(detailL.contents[0].split()[0])[0]
                                dst['age'] = list(detailL.contents[0].split()[0])[1]
                                #dst['trainerA'] = detailL.span.string.split('･')[0]
                                #dst['trainerB'] = detailL.span.string.split('･')[1]
                                if len(detailL.contents)>=2:
                                    dst['weight'] = detailL.contents[3].contents[0][:-1].split('(')[0]
                                    dst['c_weight'] = detailL.contents[3].contents[0][:-1].split('(')[1] #多分馬の体重変動
                                detailR = detail_right[m].contents
                                try:
                                    dst['jackie'] = detailR[0].strip().split(' ')[0]
                                    dst['j_weight'] = detailR[0].strip().split(' ')[1] #多分jackieの体重変動
                                    dst['trainerA'] = detailR[2].strip().split('･')[0]
                                    dst['trainerB'] = detailR[2].strip().split('･')[1]
                                except:
                                    dst['jackie'] = detailR[1].contents[0]
                                    dst['j_weight'] = float(detailR[2])
                                    dst['trainerA'] = detailR[-1].strip().split('･')[0]
                                    dst['trainerB'] = detailR[-1].strip().split('･')[1]

                                Odds = odds[m].contents[1]
                                if Odds.dt.string is not None:
                                    dst['odds'] = Odds.dt.string.strip('倍')
                                    dst['popu'] = Odds.dd.string.strip('人気') #何番人気か
                                dst['wakuban'] = waku[2*m].contents[1].contents[0]
                                dst['umaban'] = waku[2*m+1].contents[1].contents[0]
                                dst['got_money'] = money_list[int(dst['rank'])-1] if int(dst['rank']) <= len(money_list) else 0
                            except:
                                import pdb; pdb.set_trace()
                                print('passed')
                                continue
                            dst.name = str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l) + numStr(m)

                            #df = df.append(dst)
                            dic[key] = dst.to_dict()
                            key += 1
    df = df.from_dict(dic, orient='index')
    df.to_csv(f'data/keiba_PS{year}.csv')

KeyboardInterrupt: 

In [2]:
!pip install bs4

  Using cached bs4-0.0.1-py3-none-any.whl
  Using cached beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
  Using cached soupsieve-2.3.1-py3-none-any.whl (37 kB)
